# Table of contents

* [0. Preparation](#preparation)
* [1. Reading recording and sorting](#loading)
* [2. Preprocessing](#preprocessing)
* [3. Saving and loading SpikeInterface objects](#save-load)
* [4. Data compression](#compression)
* [5. Extracting waveforms](#waveforms)
* [6. Postprocessing](#postprocessing)
* [7. Validation and curation](#curation)
* [8. Viewers](#viewers)
* [9. Spike sorting comparison](#comparison)
* [10. Exporters](#exporters)

## 0. Preparation <a class="anchor" id="preparation"></a>

In [1]:
import spikeinterface as si
import spikeinterface.extractors as se 
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.postprocessing as spost
import spikeinterface.qualitymetrics as sqm
import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
import spikeinterface.widgets as sw

In [2]:
print(f"SpikeInterface version: {si.__version__}")

SpikeInterface version: 0.96.2.dev0


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import warnings
warnings.simplefilter("ignore")

%matplotlib widget

## 1. Reading recording <a class="anchor" id="loading"></a>

In [4]:
base_folder = Path(r"D:\Vincent\Data\sc011\sc011_0106\sc011_0106_001")
file_path = r"D:\Vincent\Data\sc011\sc011_0106\sc011_0106_001\Record Node 101"

### Read recording

The `extractor` module allows you to load many file formats used in electrophysiology. 

The extractors available in SI are all loaded using the [NEO](https://neo.readthedocs.io/en/stable/) python package.

We can access the full list of available extractors with:

In [5]:
se.recording_extractor_full_dict

{'binary': spikeinterface.core.binaryrecordingextractor.BinaryRecordingExtractor,
 'numpy': spikeinterface.core.numpyextractors.NumpyRecording,
 'shybrid': spikeinterface.extractors.shybridextractors.SHYBRIDRecordingExtractor,
 'mda': spikeinterface.extractors.mdaextractors.MdaRecordingExtractor,
 'nwb': spikeinterface.extractors.nwbextractors.NwbRecordingExtractor,
 'cbin_ibl': spikeinterface.extractors.cbin_ibl.CompressedBinaryIblExtractor,
 'mcsh5': spikeinterface.extractors.mcsh5extractors.MCSH5RecordingExtractor,
 'alphaomega': spikeinterface.extractors.neoextractors.alphaomega.AlphaOmegaRecordingExtractor,
 'axona': spikeinterface.extractors.neoextractors.axona.AxonaRecordingExtractor,
 'biocam': spikeinterface.extractors.neoextractors.biocam.BiocamRecordingExtractor,
 'blackrock': spikeinterface.extractors.neoextractors.blackrock.BlackrockRecordingExtractor,
 'ced': spikeinterface.extractors.neoextractors.ced.CedRecordingExtractor,
 'edf': spikeinterface.extractors.neoextractors

In [12]:
recording = se.read_openephys(file_path, stream_id='1')

In [13]:
recording

OpenEphysBinaryRecordingExtractor: 384 channels - 1 segments - 30.0kHz - 712.440s

A `RecordingExtractor` object extracts information about channel ids, channel locations (if present), the sampling frequency of the recording, and the extracellular traces (when prompted).
Here we retrieve information from the recording using the built-in functions from the RecordingExtractor:

In [ ]:
channel_ids = recording.get_channel_ids()
fs = recording.get_sampling_frequency()
num_chan = recording.get_num_channels()
num_segments = recording.get_num_segments()

print(f'Channel ids: {channel_ids}')
print(f'Sampling frequency: {fs}')
print(f'Number of channels: {num_chan}')
print(f"Number of segments: {num_segments}")

SpikeInterface supports multi-segment recordings. A segment is a contiguous piece of data, and sometimes recordings can be made of multiple acquisitions, for examples a baseline, a stimulation phase, and a post recording. In such cases, the recording object will be made of multiple segments and be treated as such over the pipeline.

The `get_traces()` function returns a TxN numpy array where N is the number of channel ids passed in (all channel ids are passed in by default) and T is the number of frames (determined by start_frame and end_frame).

In [ ]:
trace_snippet = recording.get_traces(start_frame=int(fs*0), end_frame=int(fs*2))

In [ ]:
print('Traces shape:', trace_snippet.shape)

Before moving on with the analysis, let's check the probe information. 
`SpikeInterface` internally uses an additional package, named [ProbeInterface](https://probeinterface.readthedocs.io/en/main/index.html), to handle probe devices. 

For NWB files, the probe information is loaded automatically from the file, so we can directly access its information and channel locations.

In [ ]:
probe = recording.get_probe()
print(probe)

In [ ]:
print(type(probe))

In [ ]:
sw.plot_probe_map(recording)

The `widgets` module includes several convenient plotting functions that can be used to explore the data:

In [ ]:
w_ts = sw.plot_timeseries(recording, order_channel_by_depth=True, backend="ipywidgets",
                          clim=(-200, 200))

### Properties 

`RecordingExtractor` object can have *properties*. A property is a piece of information attached to a channel, e.g. group or location.

Similarly, for `SortingExtractor` objects (that we'll cover later), anything related to a unit can be stored as a property. 

We can check which properties are in the extractor as follows:

In [ ]:
print("Properties:\n", list(recording.get_property_keys()))

Let's add some new properties! 
The first 192 channels are in the CA1 area, the second 192 are in the CA3 area:

In [ ]:
# brain_area_property_values = ['CA1']*192 + ['CA3']*192
# print(brain_area_property_values)

In [ ]:
# recording.set_property(key='brain_area', values=brain_area_property_values)

We can also specify a property on a subset of channels. In this case, the non-specified channels will be filled empty values:

In [ ]:
# recording.set_property(key='quality', values=["good"]*(recording_nwb.get_num_channels() - 3),
#                            ids=recording_nwb.get_channel_ids()[:-3])

In [ ]:
# recording.get_property("quality")[-10:]

In [ ]:
# print("Properties after adding custom properties:\n", list(recording_nwb.get_property_keys()))

**NOTE:** Internally the properties is just a dictionary attached to the recording that is accessible as `_properties`

In [ ]:
print(recording._properties.keys())

### Annotations

*Annotations* can be attached to any object and they can carry any information related to the recording or sorting objects.

Let's add an annotation about this tutorial:

In [ ]:
recording.annotate(description="First in-vivo NPX")

In [ ]:
print(recording.get_annotation_keys())

# 2. Preprocessing <a class="anchor" id="preprocessing"></a>


Now that the probe information is loaded we can do some preprocessing using `preprocessing` module.

We can filter the recordings, rereference the signals to remove noise, discard noisy channels, whiten the data, remove stimulation artifacts, etc. (more info [here](https://spiketoolkit.readthedocs.io/en/latest/preprocessing_example.html)).

For this notebook, let's filter the recordings and apply common median reference (CMR). All preprocessing modules return new `RecordingExtractor` objects that apply the underlying preprocessing function. This allows users to access the preprocessed data in the same way as the raw data.

### Phase shift

In [ ]:
recording = spre.phase_shift(recording)

### High-pass filter

In [ ]:
recording_f = spre.highpass_filter(recording, freq_min=300)
# recording = spre.bandpass_filter(recording, freq_min=300, freq_max=6000)

We can `annotate` the recording to tell SI that it is filtered. This information is useful later in the pipeline, when extracting waveforms.

In [ ]:
recording.annotate(is_filtered=False)
recording_f.annotate(is_filtered=True)

### Common Median Reference

In [ ]:
recording_cmr = spre.common_reference(recording_f, reference='global', operator='median')

We can plot the traces after applying CMR:

In [ ]:
w = sw.plot_timeseries({"raw": recording, "filt": recording_f, "common": recording_cmr},
                        clim=(-50, 50), time_range=[10, 10.1], order_channel_by_depth=True,
                        backend="ipywidgets")

# 3. Saving and loading SpikeInterface objects <a class="anchor" id="save-load"></a>

All operations in SpikeInterface are *lazy*, meaning that they are not performed if not needed. This is why the creation of our filter recording was almost instantaneous. However, to speed up further processing, we might want to **save** it to a file and perform those operations (eg. filters, CMR, etc.) at once. 

In [ ]:
job_kwargs = dict(n_jobs=10, chunk_duration="1s", progress_bar=True)

In [ ]:
if (base_folder / "preprocessed").is_dir():
    recording_saved = si.load_extractor(base_folder / "preprocessed")
else:
    recording_saved = recording_cmr.save(folder=base_folder / "preprocessed", **job_kwargs)

If we inspect the `preprocessed` folder, we find that a few files have been saved. Let's take a look at what they are:

In [ ]:
recording_saved

In [ ]:
!ls {base_folder}/preprocessed

The `traces_cached_seg0.raw` contains the processed raw data, while the `.json` files include information on how to reload the binary file. The `provenance.json` includes the information of the recording before saving it to a binary file, and the `probe.json` represents the probe object. 

The `save` returns a new *cached* recording that has all the previously loaded information:

In [ ]:
print(f'Cached channels ids: {recording_saved.get_channel_ids()}')
print(f'Channel groups after caching: {recording_saved.get_channel_groups()}')

After saving the SI object, we can easily load it back in a new session:

In [ ]:
recording_loaded = si.load_extractor(base_folder / "preprocessed")

In [ ]:
print(f'Loaded channels ids: {recording_loaded.get_channel_ids()}')
print(f'Channel groups after loading: {recording_loaded.get_channel_groups()}')

We can double check that the traces are exactly the same as the `recording_saved` that we saved:

In [ ]:
w = sw.plot_timeseries({"preprocessed": recording_cmr, "saved": recording_saved, "loaded": recording_loaded},
                        clim=(-50, 50), mode="line",
                        backend="ipywidgets")

**IMPORTANT**: the same saving mechanisms are available also for all SortingExtractor

# 4. Spike sorting <a class="anchor" id="spike-sorting"></a>

We can now run spike sorting on the above recording. We will use different spike sorters for this demonstration, to show how easy SpikeInterface makes it easy to interchengably run different sorters :)

Let's first check the available and installed sorters in `SpikeInterface`.
We will sort the bandpass cached filtered recording the `recording_saved` object.

In [ ]:
ss.available_sorters()

In [ ]:
ss.installed_sorters()

The `spikeinterface.sortingcomponents` module includes functions that can be used to create custom spike sorting pipelines built-in in `SpikeInterface`. It is still experimental and under heavy development, but there are already two SI-based sorters available:

* `tridesclous2` (developed by Samuel Garcia)
* `spykingcircus2` (developed by Pierre Yger)

They can be run with the same `run_sorter` function, but they don't require any additional installation!

We can retrieve the parameters associated to any sorter with the `get_default_params()` function from the `sorters` module:

In [ ]:
ss.get_default_sorter_params('kilosort2_5')

In [ ]:
ss.run_sorter?

To modify a parameter, we can easily pass it to the `run` function as an extra argument!

### Run sorter locally

In [ ]:
# sorter_params = job_kwargs

In [ ]:
# run spike sorting on entire recording
# sorting_KS25 = ss.run_sorter('kilosort2_5', recording_saved,
#                              output_folder=base_folder / 'results_KS25',
#                              verbose=True, **sorter_params)

In [ ]:
# print('KS25 found', len(sorting_KS25.get_unit_ids()), 'units')

In [ ]:
# sorting_KS25 = sorting_KS25.remove_empty_units()
# print(f'KS25 found {len(sorting_KS25.get_unit_ids())} non-empty units')

SpikeInterface ensures full provenance of the spike sorting pipeline. Upon running a spike sorter, a `spikeinterface_params.json` file is saved in the `output_folder`. This contains a `.json` version of the recording and all the input parameters. 

In [ ]:
# !ls {base_folder}/results_KS25

In [ ]:
# !cat {base_folder}/results_KS25/spikeinterface_params.json

The spike sorting returns a `SortingExtractor` object. Let's see some of its functions:

In [ ]:
# print(f'Spike train of a unit: {sorting_KS25.get_unit_spike_train(1)}')
# print(f'Spike train of a unit (in s): {sorting_KS25.get_unit_spike_train(1, return_times=True)}')

We can use `spikewidgets` functions for some quick visualizations:

In [ ]:
# w_rs = sw.plot_rasters(sorting_KS25)

We can also save a spike sorting output for future use:

In [ ]:
# sorting_saved_KS25 = sorting_KS25.save(folder="sorting_KS25") 

### Run sorter in container

Some sorters are hard to install! To alleviate this headache, SI provides a built-in mechanism to run a spike sorting job in a docker container.

We are maintaining a set of sorter-specific docker files in the [spikeinterface-dockerfiles repo](<https://github.com/SpikeInterface/spikeinterface-dockerfiles>)
and most of the docker images are available on Docker Hub from the [SpikeInterface organization](<https://hub.docker.com/orgs/spikeinterface/repositories>).

Running spike sorting in a docker container just requires to:

1. have docker/singularity installed
2. have docker/singularity python SDK installed (`pip install docker/spython`)

When docker/singularity is installed, you can simply run the sorter in a container image:

In [ ]:
sorter_params = job_kwargs

In [ ]:
# run spike sorting on entire recording
sorting_KS25 = ss.run_kilosort2_5(recording_saved, 
                            output_folder=base_folder / 'results_KS2_5',
                            verbose=True, docker_image=True, 
                            **sorter_params)
# Note: can't run with singularity_image on Windows because of import pwd error. Use docker_image instead.

In [ ]:
print(sorting_KS25)

In [ ]:
print(f'KS2.5 found {len(sorting_KS25.get_unit_ids())} units')

In [ ]:
sorting_KS25 = sorting_KS25.remove_empty_units()
print(f'KS2.5 found {len(sorting_KS25.get_unit_ids())} non-empty units')

# 5. Extracting waveforms <a class="anchor" id="waveforms"></a>

The core of postprocessing spike sorting results revolves around extracting waveforms from paired recording-sorting objects.

In the SI API, waveforms are extracted using the `WaveformExtractor` class in the `core` module.

The `WaveformExtractor` object has convenient functions to retrieve waveforms and templates.
Let's see how it works.

To extract the waveforms, we can run:

In [ ]:
recording_saved = si.load_extractor(base_folder / "preprocessed")
sorting = sorting_KS25
print(sorting)

In [ ]:
si.extract_waveforms?

In [ ]:
we = si.extract_waveforms(recording_saved, sorting, folder=base_folder / "waveforms", 
                          load_if_exists=False, overwrite=True, **job_kwargs)
print(we)

Now all waveforms are computed and stored in the provided `waveforms` folder. We can now retrieve waveforms and templates easily:

In [ ]:
waveforms0 = we.get_waveforms(unit_id=0)
print(f"Waveforms shape: {waveforms0.shape}")
template0 = we.get_template(unit_id=0)
print(f"Template shape: {template0.shape}")
all_templates = we.get_all_templates()
print(f"All templates shape: {all_templates.shape}")

For waveforms, the dimension is (num_spikes, num_samples, num_channels), while each template has dimension (num_samples, num_channels). Note that the number of spikes in this case is 500..we'll get back to it later!

The `WaveformExtractor` is also compatible with several `widgets` to visualize the spike sorting output:

**NEW in v0.95**: we can also render interactive plots with the `ipywidgets` backend!

In [ ]:
w = sw.plot_unit_templates(we, radius_um=30, backend="ipywidgets")

As we noticed before, the number of spikes for the waveforms is 500. Let's check the number of spikes for other waveforms:

In [ ]:
for unit in sorting.get_unit_ids():
    waveforms = we.get_waveforms(unit_id=unit)
    spiketrain = sorting.get_unit_spike_train(unit)
    print(f"Unit {unit} - num waveforms: {waveforms.shape[0]} - num spikes: {len(spiketrain)}")

No units have more than 500 spikes! This is because by default, the `WaveformExtractor` extracts waveforms on a random subset of 500 spikes. To extract waveforms on all spikes, we can use the `max_spikes_per_unit` argument:

In [ ]:
we_all = si.extract_waveforms(recording_saved, sorting, folder=base_folder / "waveforms_all", 
                              max_spikes_per_unit=None,
                              overwrite=True,
                              **job_kwargs)

In [ ]:
for unit in sorting.get_unit_ids():
    waveforms = we_all.get_waveforms(unit_id=unit)
    spiketrain = sorting.get_unit_spike_train(unit)
    print(f"Unit {unit} - num waveforms: {waveforms.shape[0]} - num spikes: {len(spiketrain)}")

Now waveforms have been extracted for all spikes! Let's move on to explore the postprocessing capabilities of the `postprocessing` module.

# 6. Postprocessing <a class="anchor" id="postprocessing"></a>

Postprocessing spike sorting results ranges from computing additional information, such as spike amplitudes and Principal Component Analisys (PCA) scores, to computing features of the extracellular waveforms, similarity between templates and crosscorrelograms. All of this is possible with the `postprocessing` module.

## Sparsity

Especially when working with silicon high-density probes, or when our probe has multiple groups (e.g. multi-shank, tetrodes), we don't care about waveform/templates on all channels. In order to find a subset of channels for each unit, we can use the `get_template_channel_sparsity()`

In [ ]:
spost.get_template_channel_sparsity?

In [ ]:
# example: radius
sparsity_radius = spost.get_template_channel_sparsity(we, method="radius", radius_um=50)
print(sparsity_radius[sorting.unit_ids[0]])

In [ ]:
# example: best
sparsity_best = spost.get_template_channel_sparsity(we, method="best_channels", num_channels=4)
print(sparsity_best[sorting.unit_ids[0]])

Most of the plotting and exporting functions accept `sparsity` as an argument. 

In [ ]:
sw.plot_unit_templates(we, sparsity=sparsity_radius, backend="ipywidgets")

In [ ]:
sw.plot_unit_templates(we, sparsity=sparsity_best, backend="ipywidgets")

### PCA scores

PCA scores can be easily computed with the `compute_principal_components()` function. Similarly to the `extract_waveforms`, the function returns an object of type `WaveformPrincipalComponent` that allows to retrieve all pc scores on demand.

In [ ]:
spost.compute_principal_components?

In [ ]:
pc = spost.compute_principal_components(we, n_components=3,
                                        sparsity=sparsity_radius, 
                                        load_if_exists=False,
                                        n_jobs=job_kwargs["n_jobs"], 
                                        progress_bar=job_kwargs["progress_bar"])

In [ ]:
pc0 = pc.get_projections(unit_id=0)
print(f"PC scores shape: {pc0.shape}")
all_labels, all_pcs = pc.get_all_projections()
print(f"All PC scores shape: {all_pcs.shape}")

For pc scores of a single unit, the dimension is (num_spikes, num_components, num_channels). 

## WaveformExtensions

When we compute PCA (or use other postprocessing functions), the computed information is added to the waveform folder. The functions act as `WaveformExtensions`:

In [ ]:
we.get_available_extension_names()

Each `WaveformExtension` is an object that allows us to retrieve the data:

In [ ]:
pc = we.load_extension("principal_components")
print(pc)

In [ ]:
all_labels, all_pcs = pc.get_data()
print(all_pcs.shape)

### Spike amplitudes

Spike amplitudes can be computed with the `get_spike_amplitudes` function.

In [ ]:
amplitudes = spost.compute_spike_amplitudes(we, outputs="by_unit", load_if_exists=True, 
                                            **job_kwargs)

By default, all amplitudes are concatenated in one array with all amplitudes form all spikes. With the `output="by_unit"` argument, instead, a dictionary is returned:

In [ ]:
amplitudes[0]

In [ ]:
sw.plot_amplitudes(we, backend="ipywidgets")

### Compute unit and spike locations

When using silicon probes, we can estimate the unit (or spike) location with triangulation. This can be done either with a simple center of mass or by assuming a monopolar model:

$$V_{ext}(\boldsymbol{r_{ext}}) = \frac{I_n}{4 \pi \sigma |\boldsymbol{r_{ext}} - \boldsymbol{r_{n}}|}$$

where $\boldsymbol{r_{n}}$ is the position of the neuron, and $\boldsymbol{r_{n}}$ of the electrode(s).

In [ ]:
unit_locations = spost.compute_unit_locations(we, method="monopolar_triangulation", load_if_exists=True)
spike_locations = spost.compute_spike_locations(we, method="center_of_mass", load_if_exists=True,
                                                **job_kwargs)

In [ ]:
sw.plot_unit_locations(we, backend="ipywidgets")

In [ ]:
sw.plot_spike_locations(we, max_spikes_per_unit=300, backend="ipywidgets")

### Compute correlograms

In [ ]:
ccgs, bins = spost.compute_correlograms(we)

In [ ]:
sw.plot_autocorrelograms(we, unit_ids=sorting.unit_ids[:3])
sw.plot_crosscorrelograms(we, unit_ids=sorting.unit_ids[:3])

### Compute template similarity

In [ ]:
similarity = spost.compute_template_similarity(we)

### Compute template metrics

Template metrics, or extracellular features, such as peak to valley duration or full-width half maximum, are important to classify neurons into putative classes (excitatory - inhibitory). The `postprocessing` allows one to compute several of these metrics:

In [ ]:
print(spost.get_template_metric_names())

In [ ]:
template_metrics = spost.calculate_template_metrics(we)
display(template_metrics)

In [ ]:
sw.plot_template_metrics(we, include_metrics=["peak_to_valley", "half_width"], 
                         backend="ipywidgets")

For more information about these template metrics, we refer to this [documentation](https://github.com/AllenInstitute/ecephys_spike_sorting/tree/master/ecephys_spike_sorting/modules/mean_waveforms) from the Allen Institute.

In [ ]:
!ls waveforms

# 7. Quality metrics and curation <a class="anchor" id="curation"></a>

The `qualitymetrics` module also provides several functions to compute qualitity metrics to validate the spike sorting results.

Let's see what metrics are available:

In [ ]:
print(sqm.get_quality_metric_list())
print(sqm.get_quality_pca_metric_list())

In [ ]:
qm = sqm.compute_quality_metrics(we, sparsity=sparsity_radius, verbose=True, 
                                 n_jobs=job_kwargs["n_jobs"])

In [ ]:
display(qm)

In [ ]:
sw.plot_quality_metrics(we, include_metrics=["amplitude_cutoff", "presence_ratio", "isi_violations_ratio", "snr"], 
                        backend="ipywidgets")

For more information about these waveform features, we refer to the [SpikeInterface documentation](https://spikeinterface.readthedocs.io/en/latest/module_qualitymetrics.html) and to this excellent [documentation](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/ecephys_quality_metrics.html) from the Allen Institute.

## Automatic curation based on quality metrics

A viable option to curate (or at least pre-curate) a spike sorting output is to filter units based on quality metrics. As we have already computed quality metrics a few lines above, we can simply filter the `qm` dataframe based on some thresholds.

Here, we'll only keep units with an ISI violation threshold < 0.2 and amplitude cutoff < 0.9:

In [ ]:
isi_viol_thresh = 0.2
amp_cutoff_thresh = 0.1

A straightforward way to filter a pandas dataframe is via the `query`.
We first define our query (make sure the names match the column names of the dataframe):

In [ ]:
our_query = f"amplitude_cutoff < {amp_cutoff_thresh} & isi_violations_rate < {isi_viol_thresh}"
print(our_query)

and then we can use the query to select units:

In [ ]:
keep_units = qm.query(our_query)
keep_unit_ids = keep_units.index.values

In [ ]:
sorting_auto_KS25 = sorting.select_units(keep_unit_ids)
print(f"Number of units before curation: {len(sorting.get_unit_ids())}")
print(f"Number of units after curation: {len(sorting_auto_KS25.get_unit_ids())}")

We can also save all the waveforms and post-processed data for curated units in a separate folder:

In [ ]:
!pip list

In [ ]:
we.select_units?

In [ ]:
we_curated = we.select_units(keep_unit_ids, new_folder="waveforms_cur")
# we_curated = we.select_units(keep_unit_ids, new_folder=None, use_relative_path="waveforms_curated")

In [ ]:
print(we_curated)

In [ ]:
we_curated.get_available_extension_names()

# 8. Viewers <a class="anchor" id="viewers"></a>


### SpikeInterface GUI

A QT-based GUI built on top of SpikeInterface objects.

Developed by Samuel Garcia, CRNL, Lyon.

In [ ]:
#!sigui waveforms/

### Sorting Summary - SortingView

The `sortingview` backend requires an additional step to configure the transfer of the data to be plotted to the cloud. 

See documentation [here](https://spikeinterface.readthedocs.io/en/latest/module_widgets.html).

Developed by Jeremy Magland and Jeff Soules, Flatiron Institute, NYC

In [ ]:
w = sw.plot_sorting_summary(we_curated, sparsity=sparsity_radius, backend="sortingview")

In [ ]:
w.view

# 9. Spike sorting comparison <a class="anchor" id="comparison"></a>

Can we combine the output of multiple sorters to curate the spike sorting output?

To answer this question we can use the `comparison` module.
We first compare and match the output spike trains of the different sorters, and we can then extract a new `SortingExtractor` with only the units in agreement.

### Compare two sorters

In [ ]:
comp_KS2_KS25 = sc.compare_two_sorters(sorting_KS2, sorting_KS25, 'KS2', 'KS25')

In [ ]:
sw.plot_agreement_matrix(comp_KS2_KS25)

In [ ]:
comp_KS2_KS25auto = sc.compare_two_sorters(sorting_KS2, sorting_auto_KS25, 'KS2', 'KS25_auto')

In [ ]:
sw.plot_agreement_matrix(comp_KS2_KS25auto)

### Compare multiple sorters

In [ ]:
mcmp = sc.compare_multiple_sorters([sorting_KS2, sorting_KS25], ['KS2', 'KS25'], 
                                   spiketrain_mode='union', verbose=True)

In [ ]:
w = sw.plot_multicomp_agreement(mcmp)
w = sw.plot_multicomp_agreement_by_sorter(mcmp)

In [ ]:
sw.plot_multicomp_graph(mcmp, draw_labels=False)

In [ ]:
agreement_sorting = mcmp.get_agreement_sorting(minimum_agreement_count=2)
print(agreement_sorting)

In [ ]:
sw.plot_rasters(agreement_sorting)

In [ ]:
# compare consensus and auto
comp_agr_auto = sc.compare_two_sorters(agreement_sorting, sorting_auto_KS25, 'AGR', 'AUTO')

In [ ]:
sw.plot_agreement_matrix(comp_agr_auto)

# 10. Exporters <a class="anchor" id="exporters"></a>

## Export to Phy for manual curation

To perform manual curation we can export the data to [Phy](https://github.com/cortex-lab/phy). 

In [ ]:
sexp.export_to_phy?

In [ ]:
sexp.export_to_phy(we, output_folder=base_folder / 'phy_KS25', 
                   compute_amplitudes=False, compute_pc_features=False, copy_binary=True,
                   **job_kwargs)

There is a problem with the latest version of Phy so we need to set an environment variable to make it work properly:

- Python:
```
import os
os.environ["QTWEBENGINE_CHROMIUM_FLAGS"] = "--single-process"
```

- OR terminal:

  - Linux/MacOS:
`export QTWEBENGINE_CHROMIUM_FLAGS="--single-process"`

  - Windows:
`set QTWEBENGINE_CHROMIUM_FLAGS="--single-process"`

Then we can run the Phy GUI:

In [ ]:
import os
os.environ["QTWEBENGINE_CHROMIUM_FLAGS"] = "--single-process"

In [ ]:
# %%capture --no-display
# !phy template-gui phy_KS25/params.py

After curating the results we can reload it using the `PhySortingExtractor` and exclude the units that we labeled as `noise`:

In [ ]:
sorting_phy_curated = se.PhySortingExtractor(base_folder / 'phy_KS25/', exclude_cluster_groups=['noise'])

In [ ]:
print(f"Number of units before curation: {len(sorting.get_unit_ids())}")
print(f"Number of units after curation: {len(sorting_phy_curated.get_unit_ids())}")

### That's all for today! Thanks!